In [1]:
import boto3

In [5]:
from boto3.dynamodb.conditions import Key

dynamodb = boto3.resource('dynamodb', region_name='', aws_access_key_id='', aws_secret_access_key='')
orders = dynamodb.Table('orders')

In [14]:
orders_det ={'order_id': 1,
        'customer_id':  101,
        'payment_method': 'Credit Card',
        'order_date':'2020-09-23'}

In [15]:
#Inserting data into dynamoDB
response = orders.put_item(Item=orders_det)

In [17]:
#reading data from dynamoDB
respone = orders.get_item(Key={"order_id":1})
print(respone)

{&#39;Item&#39;: {&#39;customer_id&#39;: Decimal(&#39;101&#39;), &#39;order_date&#39;: &#39;2020-09-23&#39;, &#39;order_id&#39;: Decimal(&#39;1&#39;), &#39;payment_method&#39;: &#39;Credit Card&#39;}, &#39;ResponseMetadata&#39;: {&#39;RequestId&#39;: &#39;9062UV281ERT8E8M0OSQT32ICVVV4KQNSO5AEMVJF66Q9ASUAAJG&#39;, &#39;HTTPStatusCode&#39;: 200, &#39;HTTPHeaders&#39;: {&#39;server&#39;: &#39;Server&#39;, &#39;date&#39;: &#39;Thu, 01 Oct 2020 16:11:53 GMT&#39;, &#39;content-type&#39;: &#39;application/x-amz-json-1.0&#39;, &#39;content-length&#39;: &#39;126&#39;, &#39;connection&#39;: &#39;keep-alive&#39;, &#39;x-amzn-requestid&#39;: &#39;9062UV281ERT8E8M0OSQT32ICVVV4KQNSO5AEMVJF66Q9ASUAAJG&#39;, &#39;x-amz-crc32&#39;: &#39;3466572613&#39;}, &#39;RetryAttempts&#39;: 0}}
